In [1]:
from PIL import Image
import os
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
import pandas as pd
from IPython.display import display, Markdown
import SimpleITK as sitk

import sys
sys.path.append('../../rse-panimg')

from pathlib import Path
from panimg import convert

In [19]:
ATTENTION_PARENT_DIR = f"V:/experiments/lung-malignancy-fairness-shaurya/nlst/sybil_attentions"
ATTENTION_GIF_DIR_DIR = f"{ATTENTION_PARENT_DIR}/attention_gifs"
ATTENTION_PNG_DIR_DIR = f"{ATTENTION_PARENT_DIR}/attention_imgs"
ATTENTION_MHA_DIR_DIR = f"{ATTENTION_PARENT_DIR}/attention_mha2d"
ATTENTION_MHA3D_DIR   = f"{ATTENTION_PARENT_DIR}/attention_mha3d_skip2d"

In [20]:
os.makedirs(ATTENTION_PNG_DIR_DIR, exist_ok=True)
os.makedirs(ATTENTION_MHA_DIR_DIR, exist_ok=True)
os.makedirs(ATTENTION_MHA3D_DIR, exist_ok=True)

In [ ]:
subfolders = [f[:-4] for f in os.listdir(ATTENTION_GIF_DIR_DIR)]

In [13]:
# for i in range(1):
#     seriesid = subfolders[i]
#     n_frames = len(os.listdir(f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"))
#     print(f"{seriesid}: {n_frames} frames")
#     result = convert(
#         input_directory=Path(f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"),
#         output_directory=Path(f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"),
#     )
#     print(result)

In [10]:
def split_gif_to_frames(gif_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    with Image.open(gif_path) as gif:
        frame_count = gif.n_frames
        print(f"Total frames: {frame_count}")

        for frame in range(frame_count):
            gif.seek(frame)
            frame_filename = os.path.join(output_folder, f"frame_{frame:03d}.png")
            gif.save(frame_filename, format="PNG")
            # print(f"Saved: {frame_filename}")

In [11]:
def stack_mha_files(input_folder, output_mha_path):
    # Get list of MHA files in the folder, sorted by filename (assuming sequential naming)
    frame_subdirs = sorted(os.listdir(input_folder))
    mha_files = [f'{sd}/{os.listdir(f"{input_folder}/{sd}")[0]}' for sd in frame_subdirs][1:]
    
    # mha_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.mha')])

    if not mha_files:
        raise ValueError("No MHA files found in the provided directory")

    # Read the first MHA file to determine the size and spacing of the slices
    first_mha_path = os.path.join(input_folder, mha_files[0])
    first_image = sitk.ReadImage(first_mha_path, outputPixelType=sitk.sitkVectorUInt8)
    slice_size = first_image.GetSize()  # (width, height, depth)
    print("slice size:", slice_size, ", num_components:", first_image.GetNumberOfComponentsPerPixel())

    # Create an empty list to hold the images
    image_list = []

    # Load each MHA file and append the images to the list
    for i, mha_file in enumerate(mha_files):
        mha_path = os.path.join(input_folder, mha_file)
        img = sitk.ReadImage(mha_path, sitk.sitkVectorUInt8)
        # print("slice ", i, " size:", img.GetSize(), ", num_components:", img.GetNumberOfComponentsPerPixel())
        if img.GetNumberOfComponentsPerPixel() == 3:
            image_list.append(img)

    # Stack the images along the third dimension (Z-axis)
    print("number of frames in stack:", len(image_list))
    stacked_image = sitk.JoinSeries(image_list)
    stack_size = stacked_image.GetSize()
    print("stack size:", stack_size, ", stack n_components:", stacked_image.GetNumberOfComponentsPerPixel())

    # Save the stacked image as a single 3D MHA file
    sitk.WriteImage(stacked_image, output_mha_path)
    print(f"Stacked MHA saved as {output_mha_path}")

In [12]:
## SKIP panimg.
def stack_png_to_mha(input_folder, output_mha_path):
    # Get list of MHA files in the folder, sorted by filename (assuming sequential naming)
    # frame_subdirs = sorted(os.listdir(input_folder))
    # mha_files = [f'{sd}/{os.listdir(f"{input_folder}/{sd}")[0]}' for sd in frame_subdirs][1:]
    
    png_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.png')])

    if not png_files:
        raise ValueError("No MHA files found in the provided directory")

    # Read the first MHA file to determine the size and spacing of the slices
    first_png_path = os.path.join(input_folder, png_files[0])
    first_image = sitk.ReadImage(first_png_path, outputPixelType=sitk.sitkVectorUInt8)
    slice_size = first_image.GetSize()  # (width, height, depth)
    print("slice size:", slice_size, ", num_components:", first_image.GetNumberOfComponentsPerPixel())

    # Create an empty list to hold the images
    image_list = []

    # Load each PNG file and append the images to the list
    for i, png_file in enumerate(png_files):
        png_path = os.path.join(input_folder, png_file)
        img = sitk.ReadImage(png_path, sitk.sitkVectorUInt8)
        # print("slice ", i, " size:", img.GetSize(), ", num_components:", img.GetNumberOfComponentsPerPixel())
        if img.GetNumberOfComponentsPerPixel() == 3:
            image_list.append(img)

    # Stack the images along the third dimension (Z-axis)
    print("number of frames in stack:", len(image_list))
    stacked_image = sitk.JoinSeries(image_list)
    stack_size = stacked_image.GetSize()
    print("stack size:", stack_size, ", stack n_components:", stacked_image.GetNumberOfComponentsPerPixel())

    # Save the stacked image as a single 3D MHA file
    sitk.WriteImage(stacked_image, output_mha_path)
    print(f"Stacked MHA saved as {output_mha_path}")

In [16]:
# for i in range(1):
#     seriesid = subfolders[i]
#     n_frames = len(os.listdir(f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"))
#     print(f"{seriesid}: {n_frames} subfolders")
#     input_folder = f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"  # Folder containing the individual MHA slices
#     output_mha_path = f"{ATTENTION_MHA3D_DIR}/{seriesid}.mha"  # Path to save the stacked MHA file
#     stack_mha_files(input_folder, output_mha_path)

In [ ]:
# for i in range(44, len(subfolders)):
for i in range(1):
    seriesid = subfolders[i]
    print(f"\n{i+1} / {len(subfolders)}: converting ...")
    
    gif_path = f"{ATTENTION_GIF_DIR_DIR}/{seriesid}.gif"
    png_folder = f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"
    split_gif_to_frames(gif_path, png_folder)
    n_frames = len(os.listdir(f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"))
    print(f"{n_frames} frames")

    # result = convert(
    #     input_directory=Path(f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"),
    #     output_directory=Path(f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"),
    # )
    # print(result)
    # n_frames = len(os.listdir(f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"))
    # print(f"{n_frames} subfolders")

    # input_folder = f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"  # Folder containing the individual MHA slices
    output_mha_path = f"{ATTENTION_MHA3D_DIR}/{seriesid}.mha"  # Path to save the stacked MHA file
    stack_png_to_mha(png_folder, output_mha_path)